In [16]:
import serial
import math
import numpy as np
import time


In [67]:
GRAVITY = np.array([[0],[0],[9.81]])
class Motion:
    def rotationMatrix(W:np.ndarray, rotMatrix:np.ndarray, delta):
        #R = np.zeros(shape=(3,3),dtype=np.float64)
        wx = W[0][0]
        wy = W[1][0]
        wz = W[2][0]
        S = np.array([
            [0,-wz,wy],
            [wz,0,-wx],
            [-wy,wx,0]
            ])
        _R = np.matmul(rotMatrix,S) * delta
        R = np.add(rotMatrix,  _R)
        return R
    def acceleration(a:np.ndarray,rotMatrix:np.ndarray,G:np.ndarray):
        return np.add(np.matmul(rotMatrix,a), G)
    def velocity(a:np.ndarray,v0:np.ndarray,delta):
        return np.add(a*delta, v0)
    def position(v:np.ndarray,p0:np.ndarray,delta):
        return np.add(v*delta ,p0)
    



R = Motion.rotationMatrix(np.array([[0],[0],[0]]),np.array([[1,0,0],[0,1,0],[0,0,1]]),0.1)
A = Motion.acceleration(np.array([[0],[0],[-9.81]]),R,GRAVITY)
print(A)

[[0.]
 [0.]
 [0.]]


In [68]:
mpu = serial.Serial(baudrate=9600,port="COM4",bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)


# Ultima calibracion
# 12:52:19.455 -> averaging 10000 readings each time
# 12:52:21.954 -> .................... [-1657,-1656] --> [-2,14]	[-1077,-1075] --> [-13,2]	[1515,1517] --> [16364,16385]	[80,81] --> [0,5]	[34,35] --> [-1,2]	[45,46] --> [0,5]
# 12:53:12.000 -> ....................	XAccel			YAccel				ZAccel			XGyro			YGyro			ZGyro
# 12:53:59.526 ->  [-1657,-1656] --> [-3,14]	[-1077,-1076] --> [-13,2]	[1516,1517] --> [16384,16385]	[80,81] --> [0,5]	[34,35] --> [-1,2]	[45,46] --> [0,5]
# 12:54:02.118 -> .................... [-1657,-1656] --> [-3,14]	[-1077,-1076] --> [-15,2]	[1516,1517] --> [16381,16385]	[80,81] --> [0,5]	[34,35] --> [-1,2]	[45,45] --> [0,1]
# 12:54:49.703 -> -------------- done --------------
offsets = np.array([
            [-1657,80],
           [-1077,34],
           [1516,45]])

#mpu.open()
class MPU:
    def __init__(self, COM: serial.Serial,offset = None) -> None:
        self.mpu = COM
        self.all_data = ""
        self.started = False
        self.ended = False
        self.accFactor = -GRAVITY[2][0]
        self.wFactor = math.pi/180
        self.accFactorRaw = 0
        self.wFactorRaw = 0
        self._accelerationRaw = np.array([[0],[0],[0]],dtype=np.int16)
        self._accelerationG = np.array([[0],[0],[0]],dtype=np.float16)
        self.acceleration = np.array([[0],[0],[0]],dtype=np.float16)

        self._angularVelocityRaw = np.array([[0],[0],[0]],dtype=np.int16)
        self._angularVelocityDeg = np.array([[0],[0],[0]],dtype=np.float16)
        self.angularVelocity = np.array([[0],[0],[0]],dtype=np.float16)
        self.minGyro = 1
        #Offsets:
        #    [AccX, GyroX]
        #    [AccY, GyroY]
        #    [AccZ, GyroZ]
        self.offsets = None
        if(offset is None):
            self.offsets = np.array([[0,0],[0,0],[0,0]])
        else:
            self.offsets = offset
    def update(self):
        while not self.ended:
            while (self.mpu.in_waiting==0):
                pass
            for j in range (0,self.mpu.in_waiting):
                raw = chr(self.mpu.read(1)[0])

                if not self.started and raw != "^":
                    continue
                elif not self.started and raw == "^":
                    self.started = True
                    continue
                elif self.started and raw == "$":
                    self.started = False
                    self.ended = True
                    continue
                if self.started:
                    string = raw
                    self.all_data += string
        #Analizar datos
        s = self.all_data.split("\r\n")
        for item in s:
            if not item.startswith(("A:","W:","X:","Y:","Z:","Wx:","Wy:","Wz:")):
                continue
            try:
                name, val = item.split(" ")
            except:
                continue
            match name:
                case "A:":
                    self.accFactorRaw = int(val) 
                case "W:":
                    self.wFactorRaw = float(val)
                case "X:":
                    self._accelerationG[0][0] = float(val) + self.offsets[0][0]
                case "Y:":
                    self._accelerationG[1][0] = float(val) + self.offsets[1][0]
                case "Z:":
                    self._accelerationG[2][0] = float(val) + self.offsets[2][0]
                case "Wx:":
                    self._angularVelocityDeg[0][0] = float(val) + self.offsets[0][1]
                    if (abs(self._angularVelocityDeg[0][0]) < self.minGyro):
                        self._angularVelocityDeg[0][0] = 0.0
                case "Wy:":
                    self._angularVelocityDeg[1][0] = float(val) + self.offsets[1][1]
                    if (abs(self._angularVelocityDeg[1][0]) < self.minGyro):
                        self._angularVelocityDeg[1][0] = 0.0
                case "Wz:":
                    self._angularVelocityDeg[2][0] = float(val) + self.offsets[2][1]
                    if (abs(self._angularVelocityDeg[2][0]) < self.minGyro):
                        self._angularVelocityDeg[2][0] = 0.0
        #convertir a m/s2 y rad/s
        
        self.acceleration = self._accelerationG * self.accFactor 
        self.angularVelocity = self._angularVelocityDeg * self.wFactor 
        
D = MPU(mpu)
start = time.time()
R0 = np.array([[1,0,0],[0,1,0],[0,0,1]])
V0 = np.array([[0],[0],[0]])
P0 = np.array([[0],[0],[0]])
print(D.accFactor)

for i in range(0,100):
    D.update()
    end = time.time()
    delta = end-start
    start=end
    R = Motion.rotationMatrix(D.angularVelocity,R0,delta)
    R0 = R
    A = Motion.acceleration(D.acceleration,R,GRAVITY)
    V = Motion.velocity(A,V0,delta)
    V0 = V
    P = Motion.position(V,P0,delta)
    P0 = P
    print(f"Acc [m/s2]:{D.acceleration}")
    print(f"W[rad/s]:{D.angularVelocity}")
    print(f"Rotation matrix: {R}")
    print(f"Acceleration Vector: {A}")
    print(f"Estimated velocity: {V}")
    print(f"Estimated position: {P}")


    
mpu.close()


-9.81


ValueError: could not convert string to float: '-2.6^'

In [69]:
mpu.close()